In [69]:
import plotly
import plotly.express as px
import plotly.graph_objects as go
import numpy as np
import pandas as pd
import requests
from pathlib import Path

In [78]:
CLEANPATH = Path.cwd().parent / 'clean-data' 
PLOTPATH = Path.cwd().parent / 'plots'
HTMLPATH = Path.cwd().parent / 'plots' / 'html'
nl_geojson=r"the-netherlands.geojson"
nl_geojson2=r"/Users/srdpsrknth/OneDrive/CovidDash/covid19dash/external/the-netherlands.geojson"
nl_geojson3=r"the-netherlands.json"
nl_geojson4=r"/Users/srdpsrknth/OneDrive/CovidDash/covid19dash/plot-scripts/the-netherlands.json"

In [71]:
select = {'cumu':'Cumulative', 'daily':'Daily','total':'Confirmed', 'admitted':'Hospitalised', 'deaths':'Deaths'}

In [72]:
df_cumu = pd.read_csv(CLEANPATH /'province_cumulative_clean.csv',index_col=0)
df_daily = pd.read_csv(CLEANPATH /'province_daily_clean.csv',index_col=0)

In [73]:
province_list = df_cumu['province'].unique()
prov_series=pd.Series(province_list)

In [61]:
def province_line_plot(dtype = 'cumu', param = 'total'):    
    if dtype == 'cumu':
        df = df_cumu
    elif dtype == 'daily':
        df = df_daily
    else:
        print("Incorrect input type. Choose dtype = 'cumu' or 'daily'")
        return
    
    fig=go.Figure()

    for prov in province_list:
        fig.add_trace(
            go.Scatter(
                x=df.query("province=='{}'".format(prov))['date'],
                y=df.query("province=='{}'".format(prov))[param],
                name = prov
            )
        )

    button_all = dict(label = 'All',
                          method = 'update',
                          args = [{'visible': ((df['province'].unique())==(province_list)),
                                   'title': 'All',
                                   'showlegend':True}])    

    def create_layout_button(prov):
            return dict(label = prov,
                        method = 'update',
                        args = [{'visible': [p == prov for p in province_list],
                                 'title': prov,
                                 'showlegend': True}])

    fig.update_layout(autosize=True,
                      template="plotly_white",
                      title = "{} - {}".format(select[dtype], select[param])
                     )

    fig.update_layout(
            updatemenus=[go.layout.Updatemenu(
                active = 0,
                buttons = [button_all] + list(prov_series.map(lambda prov: create_layout_button(prov)))
                )
            ])

    fig.write_image("{}".format(PLOTPATH / 'province_{}_{}.png'.format(select[dtype],select[param])),
                    engine="kaleido", width=1920, height=1080)
    fig.write_html("{}".format(HTMLPATH / 'province_{}_{}.html'.format(select[dtype],select[param])))    

    #fig.show()

In [62]:
def province_facet(dtype = 'cumu', param = 'total'):    
    if dtype == 'cumu':
        s = 'Cumulative'
        df = df_cumu
    elif dtype == 'daily':
        s = 'Daily'
        df = df_daily
    else:
        print("Incorrect input type. Choose dtype = 'cumu' or 'daily'")
        return
    
    date_latest = df['date'].max()
    df_latest = df[df['date']==date_latest]
    

    fig = px.line(x=df['date'], y=df[param], facet_col=df['province'],facet_col_wrap=3,color=df['province'])
    fig.for_each_annotation(lambda a: a.update(text=a.text.split("=")[-1]))

    fig.update_layout(
        autosize=True,
        title = "{} Facet Plots per province - {}".format(select[dtype], select[param]),
        template="plotly_white")

    fig.write_image("{}".format(PLOTPATH / 'province_facet_{}_{}.png'.format(select[dtype],select[param])), engine="kaleido",width=1920, height=1080)
    fig.write_html("{}".format(HTMLPATH / 'province_facet_{}_{}.html'.format(select[dtype],select[param])))    

    #fig.show()

In [63]:
def nl_choropleth_animated(df, param, splits=10):
    date_start = df['date'].min()
    date_end = df['date'].max()
    r_min = df[param].min()
    r_max = df[param].max()
    
    dates = pd.date_range(start=date_start, end=date_end,periods=splits).strftime("%Y-%m-%d")
    
    fig = px.choropleth_mapbox(df[df['date'].isin(dates)], geojson=nl_geojson,
                            locations='province', color=param,
                            featureidkey="properties.name",
                            color_continuous_scale="YlOrRd",
                            range_color=(r_min,r_max),
                            mapbox_style="light",
                            zoom=6,
                            center = {"lat": 52.1326, "lon": 5.2913},
                            opacity=0.7,
                            animation_frame=df[df['date'].isin(dates)]['date']
                           )

    fig.update_layout(
        width=1920,
        height=1080,
        autosize=True,
        margin=dict(t=20, b=0, l=0, r=0)
    )

    fig.update_layout(mapbox_accesstoken="pk.eyJ1IjoiendlaWxvdXMiLCJhIjoiY2tpaXdmZ3F4MTJ4djJwbXF5dnozeHk0eiJ9.HrPRH33LUQagKn4e_fd3oQ")
    fig.write_html('province_choropleth.html') 
    #py.offline.plot(fig3, filename='test3.html')
    #fig.show()

In [89]:
def nl_choropleth_latest(dtype = 'cumu', param = 'total'):
    if dtype == 'cumu':
        df = df_cumu
    elif dtype == 'daily':
        df = df_daily
    else:
        print("Incorrect input type. Choose dtype = 'cumu' or 'daily'")
        return
    
    
    date_latest = df['date'].max()
    
    
    df_latest = df[df['date']==date_latest]
    r_min = df_latest[param].min()
    r_max = df_latest[param].max()
    
    fig = px.choropleth_mapbox(df_latest, geojson=nl_geojson4,
                            locations='province', color=param,
                            featureidkey="properties.name",
                            color_continuous_scale="YlOrRd",
                            range_color=(r_min,r_max),
                            mapbox_style="light",
                            zoom=7,
                            center = {"lat": 52.1326, "lon": 5.2913},
                            opacity=0.7
                           )

    fig.update_layout(
        autosize=True,
        margin=dict(t=20, b=0, l=0, r=0)
    )

    fig.update_layout(mapbox_accesstoken="pk.eyJ1IjoiendlaWxvdXMiLCJhIjoiY2tpaXdmZ3F4MTJ4djJwbXF5dnozeHk0eiJ9.HrPRH33LUQagKn4e_fd3oQ") 
    
    fig.write_image("{}".format(PLOTPATH / 'nl_choropleth_{}_{}.png'.format(select[dtype],select[param])), width=1920, height=1080)
    #fig.write_html("{}".format(HTMLPATH / 'nl_choropleth_{}_{}.html'.format(select[dtype],select[param])))  
    #fig.show()

In [65]:
def province_choropleth_facet(dtype = 'cumu', param = 'total'):    
    if dtype == 'cumu':
        s = 'Cumulative'
        df = df_cumu
    elif dtype == 'daily':
        s = 'Daily'
        df = df_daily
    else:
        print("Incorrect input type. Choose dtype = 'cumu' or 'daily'")
        return
    
    date_latest = df['date'].max()
    df_latest = df[df['date']==date_latest]
    df_latest = df_latest.melt(id_vars = ['date','province'], value_vars = ['total','admitted','deaths'])

    fig = px.choropleth(df_latest, geojson=nl_geojson,
                            locations='province', color='value',
                            featureidkey="properties.name",
                            color_continuous_scale="YlOrRd",
                            facet_col='variable',
                            scope='europe',
                            projection="mercator"
                           )
    
    fig.update_geos(fitbounds="locations", visible=True)
    
    fig.update_layout(
        autosize=True,
        margin=dict(t=20, b=0, l=0, r=0)
    )

    #fig.write_image("{}".format(PLOTPATH / 'province_facet_{}_{}.png'.format(dtype,param)), engine="kaleido",width=1920, height=1080)
    #fig.write_html("{}".format(HTMLPATH / 'province_facet_{}_{}.html'.format(dtype,param)))    

    #fig.show()

In [66]:
## GET LINE PLOTS FOR TOTAL RECORDED CASES, TOTAL ADMITTED, DEATHS - PROVINCIAL - CUMULATIVE###
province_line_plot(dtype='cumu', param='total')
province_line_plot(dtype='cumu', param='admitted')
province_line_plot(dtype='cumu', param='deaths')

## GET LINE PLOTS FOR TOTAL RECORDED CASES, TOTAL ADMITTED, DEATHS - PROVINCIAL - DAILY REPORTED###
province_line_plot(dtype='daily', param='total')
province_line_plot(dtype='daily', param='admitted')
province_line_plot(dtype='daily', param='deaths')

## GET FACET PLOTS FOR TOTAL RECORDED CASES, TOTAL ADMITTED, DEATHS - PROVINCIAL - CUMULATIVE###
province_facet(dtype='cumu', param='total')
province_facet(dtype='cumu', param='admitted')
province_facet(dtype='cumu', param='deaths')

## GET FACET PLOTS FOR TOTAL RECORDED CASES, TOTAL ADMITTED, DEATHS - PROVINCIAL - DAILY REPORTED###
province_facet(dtype='daily', param='total')
province_facet(dtype='daily', param='admitted')
province_facet(dtype='daily', param='deaths')

In [90]:
## GENERATE LATEST CHOROPLETH PLOT FOR NL PROVINCES - DAILY - confirmed, admitted, deaths
nl_choropleth_latest(dtype='daily', param='total')
nl_choropleth_latest(dtype='daily', param='admitted')
nl_choropleth_latest(dtype='daily', param='deaths')